_（以下用缩写 sd 表示 \[Base64\](c3RhYmxlLWRpZmZ1c2lvbg==)。）_

# SD自由

文字说明较详细的精简版：[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1RKn-X04sf8VtxnUECxQyPnYKOMw0j3P0)

本笔记本属于 [这个 github 项目](https://github.com/adpitacor/sd-webui-on-colab)。如果发现本笔记本的问题（如注释有误，错别字等）或运行中遇到问题，或者 colab 版和仓库版没有同步更新，又或者想提点建议，欢迎来提 issue。

温馨提示：

- 请在 `Edit`（修改）-> `Notebook settings`（笔记本设置）中，把 `Hareware acceleterator`（硬件加速器）选项设置为 `GPU`。

- 不要用 `Run All`（全部运行），请至少粗略阅读单元格附加的文字说明再运行。

- 单元格运行中途报错都是意料之中，判断运行有问题的唯一标准是单元格结束运行而未清空输出并留下简短的运行完成提示句（对于 3. 则是单元格在未手动结束运行自动终止）。

- 单元格 3. 启动后会持续运行，若启动后希望运行其他单元格必须先手动停止该单元格的运行。

希望本笔记本可以让你实现一定程度上的“SD自由”。

In [ ]:
#@title ## 0. 前置配置

#@markdown 本单元格将进行一些必要的设置，以做好安装 webui 的准备。
#@markdown
#@markdown 大致运行内容：
#@markdown
#@markdown 1. 安装「安装 webui」所需的包（主要是 venv，其他应该本来就有）；
#@markdown
#@markdown 1. 添加非 root 用户并适当放权，供运行 webui.sh 使用。

from IPython.display import clear_output as cls
%cd /content

!apt install -y -qq wget git python3 python3-venv

!useradd guest0721
!mkdir /home/guest0721
!chmod 777 /home -R
!chmod 777 /content -R
!chmod 777 /tmp -R
!chmod 777 /var -R 2>/dev/null

%cd /content
cls()
!echo "Finished."

In [ ]:
#@title ## 1. 预运行 webui.sh

#@markdown 本单元格将预运行 `webui.sh` 拉取 webui 包并进行一些设置，确保接下来本地的 `webui.sh` 可以顺利运行并开启 webui 服务。
#@markdown
#@markdown 大致运行内容：
#@markdown
#@markdown 1. 获取 github 上的 webui.sh 并在本地运行（只能做到拉取 webui 包）；
#@markdown
#@markdown 1. 调整 venv 并为 launch.py 启用 --share 命令行参数。

from IPython.display import clear_output as cls
%cd /content

!>  /tmp/cmd.txt
!>> /tmp/cmd.txt echo 'bash -c "bash <(wget -qO- https://raw.githubusercontent.com/AUTOMATIC1111/sta''ble-diff''usion-web''ui/master/webui.sh)"'
!>> /tmp/cmd.txt echo "mv sta""ble-dif""fusion-web""ui webui"
!>> /tmp/cmd.txt echo "cd webui"
!>> /tmp/cmd.txt echo "wget -qO /tmp/get-pip.py https://bootstrap.pypa.io/get-pip.py"
!>> /tmp/cmd.txt echo "venv/bin/python3 /tmp/get-pip.py"
!>> /tmp/cmd.txt echo "python3 -m venv venv/ --without-pip"
!>> /tmp/cmd.txt echo "sed -i 's/#export COMMANDLINE_ARGS=\"\"/export COMMANDLINE_ARGS=\"--share\"/g' webui-user.sh"
!>> /tmp/cmd.txt echo "exit"

!<  /tmp/cmd.txt su guest0721

%cd /content
cls()
!echo "Terminated."

In [ ]:
#@title ## 2.（可选，但不完全可选）下载大模型

#@markdown 本单元块旨在预先下载大模型。
#@markdown
#@markdown 当然，要安装 LoRA、插件之类的也请在这里先安装。

from IPython.display import clear_output as cls
%cd /content/webui/models

!apt install -y -qq aria2

!>  /tmp/uri.txt
!>> /tmp/uri.txt echo "https://civitai.com/api/download/models/30163"
!>> /tmp/uri.txt echo "  out=AnythingV5Ink_v5PrtRE.safetensors"
!>> /tmp/uri.txt echo "https://huggingface.co/ckpt/anything-v4.5-vae-swapped/resolve/main/anything-v4.5-vae-swapped.safetensors"
!>> /tmp/uri.txt echo "  out=anything-v4.5-vae-swapped.safetensors"
!>> /tmp/uri.txt echo "https://huggingface.co/PWB/starlake/resolve/main/starlake-3.5.safetensors"
!>> /tmp/uri.txt echo "  out=starlake-3.5.safetensors"
!>> /tmp/uri.txt echo "https://huggingface.co/gsdf/Counterfeit-V3.0/resolve/main/Counterfeit-V3.0_fp16.safetensors"
!>> /tmp/uri.txt echo "  out=Counterfeit-V3.0_fp16.safetensors"
!aria2c -i /tmp/uri.txt --summary-interval=5 -x 3 --allow-overwrite=true -d Stable-diffusion

!>  /tmp/uri.txt
!>> /tmp/uri.txt echo "https://huggingface.co/a1079602570/animefull-final-pruned/resolve/main/animevae.pt"
!>> /tmp/uri.txt echo "  out=animevae.pt"
!aria2c -i /tmp/uri.txt --summary-interval=5 -x 3 --allow-overwrite=true -d VAE

# 要安装其他诸如 LoRA、插件之类的请在此手动添加代码安装

%cd /content
cls()
!echo "Finished."

In [ ]:
#@title ## 3. 启动 webui

#@markdown 本单元格将运行本地的 webui.sh，正式启动 webui。
#@markdown
#@markdown 请耐心等待，等到 `Running on public URL: https://*.gradio.live` 一行输出后，复制链接（此即公链）并访问，即可打开 webui。国内也可访问此公链。
#@markdown
#@markdown 初次运行需要安装所需的模块，用时相对较长，属正常现象。
#@markdown
#@markdown 注意，请在运行前确认所需的模型、插件等都已安装。

from IPython.display import clear_output as cls
%cd /content/webui

!>  /tmp/cmd.txt
!>> /tmp/cmd.txt echo "./webui.sh"
!>> /tmp/cmd.txt echo "exit"

!<  /tmp/cmd.txt su guest0721

%cd /content
cls()
!echo "Terminated."

In [ ]:
#@title ## 4.（可选）把所有生成出来的图片打包带走

#@markdown 本单元格提供了一种下载已生成的图片的方式。
#@markdown
#@markdown 单元格运行完成后会输出一个链接，访问该链接即可在第三方文件托管平台（[gofile.io](https://gofile.io/)，国内可访问）下载到图片压缩包。
#@markdown
#@markdown 当然，你也可以在 webui 中保存图片，而且那样做风险或许小些——要是笔记本连接真的被强制掐断，文件就都没了。

from IPython.display import clear_output as cls
%cd /content

!rm -f images.zip
!zip -r images.zip webui/outputs

from subprocess import check_output
from json import loads
try:
  res = loads(check_output(["curl", "https://api.gofile.io/getServer"]))
  assert res["status"] == "ok"
  srv = res["data"]["server"]
  res = loads(check_output(["curl", "-F", "file=@images.zip", f"https://{srv}.gofile.io/uploadFile"]))
  assert res["status"] == "ok"
  url = res["data"]["downloadPage"]
except:
  ok = False
else:
  ok = True

%cd /content
cls()
if ok: print("link:", url)
else:  print("Failed to upload images.zip")